In [1]:
import pickle
import hddCRP.behaviorDataHandlers
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:

num_runs = 4;
runs = range(num_runs);
num_warmup_samples = 5000
num_samples        = 10000
overwrite_existing_results = True

sequential_distances_only = True # if false, model setup more as a "smoother". If true, can simulate from the model

if(sequential_distances_only):
    results_directory = "Results/sequentialModel/"
else:
    results_directory = "Results"
if(not os.path.exists(results_directory)):
    os.makedirs(results_directory)

data_filename = 'data/Data_turns_all_by_session.pkl';
with open(data_filename, 'rb') as data_file:
    data = pickle.load(data_file)

subjects = list(data["data"].keys())
subjects = ["A1"]
subjects.sort()
print("subjects = " + str(subjects))

subjects = ['A1']


In [3]:

for run_idx in runs:
    for subject_idx, subject in enumerate(subjects):
        filename = "{results_directory}/Subject_{subject_id}_run_{run_idx}.pkl".format(results_directory=results_directory, subject_id=subject, run_idx=run_idx)
        if(not os.path.isfile(filename) or overwrite_existing_results):
            # for each run, should do some randomization of initial parameters (with known seeds so we can repeat everything)

            seed = subject_idx * 1000 + run_idx;
            rng = np.random.Generator(np.random.MT19937(seed))

            sequences = data["data"][subject]["data"]; # turns in each session
            session_types = data["data"][subject]["task"] # which maze

            ii = session_types.index("C")
            sequences = [sequences[ii]]
            session_types = [session_types[ii]]


            model = hddCRP.behaviorDataHandlers.create_hddCRP(sequences, session_types, rng = rng, sequential_distances_only=sequential_distances_only)
            raise RuntimeError("stopping")
            tau_names = [str(xx) for xx in model.weight_param_labels]
            alphas_names = ["alpha_concentration_no_context", "alpha_concentration_one_back_context", "alpha_concentration_two_back_context"]
            model, samples, step_size_settings = hddCRP.behaviorDataHandlers.sample_model_for_maze_data(model, num_samples=num_samples, num_warmup_samples=num_warmup_samples)
            
            MCMC_info = {"step_size_settings" : step_size_settings.to_dict(),
                        "num_warmup_samples" : num_warmup_samples,
                        "num_samples" : num_samples,
                        "seed" : seed, 
                        "sequential_distances_only" : sequential_distances_only}
            samples["tau_parameter_names"] = tau_names
            samples["alphas_names"] = alphas_names
            
            # save results to filename
            with open(filename, "wb") as results_file:
                results_data = {"MCMC_info" : MCMC_info,
                                "samples" : samples}
                pickle.dump(results_data, results_file)

RuntimeError: stopping

In [ ]:
# example for loading results and computing estimates with error bars for a single subject
subject = 'A1'


taus = np.zeros((10000,2,4))
alphas = np.zeros((10000,3,4))

for run_idx in range(4):
    filename = "{results_directory}/Subject_{subject_id}_run_{run_idx}.pkl".format(results_directory=results_directory, subject_id=subject, run_idx=run_idx)
    with open(filename, "rb") as file:
        results = pickle.load(file)

        ss = range(results["MCMC_info"]["num_warmup_samples"], results["MCMC_info"]["num_warmup_samples"]+results["MCMC_info"]["num_samples"])
        taus[:,:,run_idx] = np.exp(results["samples"]["log_taus"][ss,:]).squeeze()
        alphas[:,:,run_idx] = results["samples"]["alphas"][ss,:]

        # ci_range = [2.5, 97.5]

        # mean_taus = np.mean(taus, axis=0)
        # mean_alphas = np.mean(alphas, axis=0)
        # std_taus = np.std(taus, axis=0)
        # std_alphas = np.std(alphas, axis=0)
        # ci95_taus = np.percentile(taus, ci_range, axis=0)
        # ci95_alphas = np.percentile(alphas, ci_range, axis=0)

        # accepted = results["samples"]["accepted"][ss]
        # print("Fraction of accepted MH samples: " + str(np.mean(accepted)))

    # plt.subplot(1,2,1)
    # plt.plot(taus)

    # plt.subplot(1,2,2)
    # plt.plot(alphas)
    # plt.show()


In [58]:
from itertools import product
print(model._groupings[:5,:])
# print(model._Y)

prefixes = [str(xx) + '-' for xx in np.unique(model._Y)]
combinations = list(product(prefixes, prefixes))

level_0_grps = ['' for xx in combinations]
level_1_grps = [''.join(xx[-1:]) for xx in combinations]
level_2_grps = [''.join(xx[-2:]) for xx in combinations]

print(level_0_grps)
print(level_1_grps)
print(level_2_grps)


g_ns_level_0 = [int(model._groupings_compact[model._groupings[:,0] == xx,0][0]) if np.any(np.isin(xx,model._groupings[:,0])) else model.num_groups[0] for xx in level_0_grps]
g_ns_level_1 = [int(model._groupings_compact[model._groupings[:,1] == xx,1][0]) if np.any(np.isin(xx,model._groupings[:,1])) else model.num_groups[1] for xx in level_1_grps]
g_ns_level_2 = [int(model._groupings_compact[model._groupings[:,2] == xx,2][0]) if np.any(np.isin(xx,model._groupings[:,2])) else model.num_groups[2] for xx in level_2_grps]

gnums = [g_ns_level_0, g_ns_level_1, g_ns_level_2]
print(g_ns_level_0)
print(g_ns_level_1)
print(g_ns_level_2)

[['' 'NULL-' 'NULL-NULL-']
 ['' '2-' 'NULL-2-']
 ['' '2-' '2-2-']
 ['' '0-' '2-0-']
 ['' '0-' '0-0-']]
['', '', '', '', '', '', '', '', '']
['0-', '1-', '2-', '0-', '1-', '2-', '0-', '1-', '2-']
['0-0-', '0-1-', '0-2-', '1-0-', '1-1-', '1-2-', '2-0-', '2-1-', '2-2-']
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 2, 0, 1, 2, 0, 1, 2]
[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [8]:
from hddCRP.modelFitting import complete_exponential_distance_function_for_maze_task, hddCRPModel


obs_idx = model.N-1
n_obs = np.size(obs_idx)

D_new = np.array(model._D[obs_idx,:,:].reshape((n_obs,model.N,1)))
inds_new = model._weight_function_setup["inds"][obs_idx,:].reshape((n_obs,model.N))
D_new[inds_new == 0] += 1

weights = complete_exponential_distance_function_for_maze_task(D_new, model.weight_params, inds_new, model._weight_function_setup["timescale_inds"],model._weight_function_setup["constant_scale_inds"])
#D_new.shape
# inds_new
print(weights)

[[0.07098156 0.07468594 0.07858365 0.08268478 0.08699994 0.09154029
  0.0963176  0.10134422 0.10663318 0.11219815 0.11805355 0.12421454
  0.13069705 0.13751787 0.14469465 0.15224598 0.1601914  0.16855147
  0.17734784 0.18660328 0.19634173 0.20658842 0.21736986 0.22871397
  0.2406501  0.25320915 0.26642364 0.28032776 0.29495752 0.31035077
  0.32654736 0.34358922 0.36152047 0.38038751 0.40023918 0.42112687
  0.44310465 0.46622941 0.490561   0.51616241 0.54309991 0.57144323
  0.60126572 0.63264459 0.66566106 0.7004006  0.73695313 0.77541326
  0.81588055 0.85845974 0.90326106 0.95040047]]


array([[0.3879662 , 0.26683665, 0.34519715],
       [0.40702903, 0.3007966 , 0.29217437],
       [0.54159123, 0.26896808, 0.18944069],
       [0.40292722, 0.27758304, 0.31948974],
       [0.39086562, 0.26949936, 0.33963502],
       [0.55544724, 0.23774744, 0.20680532],
       [0.41164383, 0.22013184, 0.36822433],
       [0.37609019, 0.33948327, 0.28442653],
       [0.51590451, 0.25889495, 0.22520054]])

array([[0.3879662 , 0.26683665, 0.34519715],
       [0.40702903, 0.3007966 , 0.29217437],
       [0.54159123, 0.26896808, 0.18944069],
       [0.40292722, 0.27758304, 0.31948974],
       [0.39086562, 0.26949936, 0.33963502],
       [0.55544724, 0.23774744, 0.20680532],
       [0.41164383, 0.22013184, 0.36822433],
       [0.37609019, 0.33948327, 0.28442653],
       [0.51590451, 0.25889495, 0.22520054]])